<a href="https://colab.research.google.com/github/Arcaneless/GAR_Efficient_Net_B0/blob/main/EfficientNet_GAR_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
!cp '/content/drive/My Drive/Colab Notebooks/socif/train.zip' /content
!cd /content/
!unzip train.zip

Archive:  train.zip
   creating: train/
  inflating: train/0001.jpg          
  inflating: train/0002.jpg          
  inflating: train/0003.jpg          
  inflating: train/0004.jpg          
  inflating: train/0005.jpg          
  inflating: train/0006.jpg          
  inflating: train/0007.jpg          
  inflating: train/0008.jpg          
  inflating: train/0009.jpg          
  inflating: train/0010.jpg          
  inflating: train/train.csv         


In [2]:
#%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from pathlib import Path
import multiprocessing
import os
import tensorflow.keras as keras

#tf.enable_control_flow_v2()
#tf.enable_eager_execution()

# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'


In [ ]:
# just double check the version
tf.__version__

'1.15.2'

In [11]:
# Building the model
efficient_net_lite_0 = "https://tfhub.dev/tensorflow/efficientnet/lite0/classification/1" 
efficient_net_lite_0_i8 = "https://tfhub.dev/tensorflow/lite-model/efficientnet/lite0/int8/1"

# Input layer
input = tf.keras.layers.Input(shape=(224,224,3), name='input_dllm')
# Pre-trained layer
eff = hub.KerasLayer(efficient_net_lite_0, trainable=False)(input)
# This output is for gender
dense_G = tf.keras.layers.Dense(units=2, activation="softmax", name="pred_gender")(eff)
# This output is for age
dense_A = tf.keras.layers.Dense(units=101, activation="softmax", name="pred_age")(eff)
# This output is for race
dense_R = tf.keras.layers.Dense(units=3, activation="softmax", name="pred_race")(eff)
model = keras.Model(inputs=input, outputs=[dense_G, dense_A, dense_R], name='HIURMOM')

In [ ]:
model.summary()

Model: "HIURMOM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_dllm (InputLayer)         [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      (None, 1000)         4694024     input_dllm[0][0]                 
__________________________________________________________________________________________________
pred_gender (Dense)             (None, 2)            2002        keras_layer_1[0][0]              
__________________________________________________________________________________________________
pred_age (Dense)                (None, 101)          101101      keras_layer_1[0][0]              
____________________________________________________________________________________________

In [12]:
from tensorflow.keras.optimizers import SGD, Adam
def get_optimizer(name="adam", learning_rate=0.001):
    if name == "sgd":
        return SGD(lr=learning_rate, momentum=0.9, nesterov=True)
    elif name == "adam":
        return Adam(lr=learning_rate)
    else:
        raise ValueError("optimizer name should be 'sgd' or 'adam'")


def get_scheduler(epoches=30, learning_rate=0.001):
    class Schedule:
        def __init__(self, nb_epochs, initial_lr):
            self.epochs = nb_epochs
            self.initial_lr = initial_lr

        def __call__(self, epoch_idx):
            if epoch_idx < self.epochs * 0.25:
                return self.initial_lr
            elif epoch_idx < self.epochs * 0.50:
                return self.initial_lr * 0.2
            elif epoch_idx < self.epochs * 0.75:
                return self.initial_lr * 0.04
            return self.initial_lr * 0.008
    return Schedule(epoches, learning_rate)

In [15]:
csv_path = "./train/train.csv"
dataset = pd.read_csv(csv_path, header=None)
X = dataset[0].apply(lambda p: cv2.imread("./train/"+p))
X = X.apply(lambda img: cv2.resize(img, (224, 224)))
X = X.apply(lambda img: cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
X = np.array(X.to_list(), dtype=np.float16)

In [17]:
y = dataset.loc[:,1:].to_numpy()

In [18]:
import tensorflow.keras as keras

# Compiling the model
opt = get_optimizer()
scheduler = get_scheduler()
model.compile(optimizer=opt, loss=["sparse_categorical_crossentropy", 
                "sparse_categorical_crossentropy",
                "sparse_categorical_crossentropy"],metrics=['accuracy'])

checkpoint_dir = Path("./checkpoint")
checkpoint_dir.mkdir(exist_ok=True)
filename = "_".join(["EfficientNetB3","224","weights.{epoch:02d}-{loss:.2f}.hdf5"])
callbacks= [
    LearningRateScheduler(schedule=scheduler),
    ModelCheckpoint(str(checkpoint_dir) + "/" + filename,
                    monitor="loss",
                    verbose=1,
                    save_best_only=True,
                    mode="auto"),
]

In [19]:
model.fit(X, y=[y[:,0], y[:,1], y[:,2]], epochs=10, batch_size=10, callbacks=callbacks)

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 9.2591 - pred_gender_loss: 1.4904 - pred_age_loss: 6.3725 - pred_race_loss: 1.3962 - pred_gender_accuracy: 0.6000 - pred_age_accuracy: 0.0000e+00 - pred_race_accuracy: 0.4000
Epoch 00001: loss improved from inf to 9.25908, saving model to checkpoint/EfficientNetB3_224_weights.01-9.26.hdf5
1/1 [==============================] - 0s 134ms/step - loss: 9.2591 - pred_gender_loss: 1.4904 - pred_age_loss: 6.3725 - pred_race_loss: 1.3962 - pred_gender_accuracy: 0.6000 - pred_age_accuracy: 0.0000e+00 - pred_race_accuracy: 0.4000
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 6.6679 - pred_gender_loss: 0.9964 - pred_age_loss: 4.5568 - pred_race_loss: 1.1147 - pred_gender_accuracy: 0.5000 - pred_age_accuracy: 0.0000e+00 - pred_race_accuracy: 0.3000
Epoch 00002: loss improved from 9.25908 to 6.66789, saving model to checkpoint/EfficientNetB3_224_weights.02-6.67.hdf5
1/1 [==============================] - 0s 131ms/

In [ ]:
model.summary()

Model: "HIURMOM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_dllm (InputLayer)         [(None, 1000)]       0                                            
__________________________________________________________________________________________________
pred_gender (Dense)             (None, 2)            2002        input_dllm[0][0]                 
__________________________________________________________________________________________________
pred_age (Dense)                (None, 101)          101101      input_dllm[0][0]                 
__________________________________________________________________________________________________
pred_race (Dense)               (None, 3)            3003        input_dllm[0][0]                 
Total params: 106,106
Trainable params: 106,106
Non-trainable params: 0
____________________

In [ ]:
from tensorflow import keras
# check input and output names
in_names = [x.op.name for x in model.inputs]
out_names = [x.op.name for x in model.outputs]
print(in_names)
print(out_names)

['input_dllm_1']
['pred_gender/Softmax', 'pred_age/Softmax', 'pred_race/Softmax']


In [ ]:
print(np.array([X.astype(np.float32)[0]]).shape)

(1, 224, 224, 3)


In [20]:
import keras.backend as K

In [21]:
# Saved the model to .h5 format, P.S. the model must not contain subclass, it should be functional or sequential
tf.keras.models.save_model(model, '/content/saved_model.h5')

In [22]:
# No Quantize version
# Convert .h5 file to .tflite
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(
    model_file='/content/saved_model.h5',
    #input_arrays=['input_dllm'],
    #output_arrays=['pred_gender/Softmax', 'pred_age/Softmax', 'pred_race/Softmax'],
    input_shapes={"input_dllm" : [None, 224,224,3]},
    custom_objects={'KerasLayer': hub.KerasLayer}
)
tflite_model = converter.convert()
open("b0_model_lite.tflite", "wb").write(tflite_model)

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


...

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


19003680

In [ ]:
# Quantize version
# Convert .h5 file to .tflite
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file(
    model_file='/content/saved_model.h5',
    #input_arrays=['input_dllm'],
    #output_arrays=['pred_gender/Softmax', 'pred_age/Softmax', 'pred_race/Softmax'],
    input_shapes={"input_dllm" : [None, 224, 224, 3]},
    custom_objects={'KerasLayer': hub.KerasLayer}
)
converter.optimizations = [tf.compat.v1.lite.Optimize.DEFAULT]
def representative_dataset_gen():
    img = tf.data.Dataset.from_tensor_slices(X.astype(np.float32)).batch(1)
    for i in img.take(10):
        yield [i]
converter.target_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS,tf.compat.v1.lite.OpsSet.SELECT_TF_OPS]
converter.inference_type = tf.compat.v1.lite.constants.QUANTIZED_UINT8
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.compat.v1.lite.OpsSet.TFLITE_BUILTINS]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()
open("b0_model_lite_qed.tflite", "wb").write(tflite_model)

/usr/local/lib/python3.6/dist-packages/tensorflow/lite/python/lite.py:1188: UserWarning: Property target_ops is deprecated, please use target_spec.supported_ops instead.
  "target_spec.supported_ops instead." % name)


INFO:tensorflow:Assets written to: /tmp/tmps3l3n2mt/assets


INFO:tensorflow:Assets written to: /tmp/tmps3l3n2mt/assets


INFO:tensorflow:Restoring parameters from /tmp/tmps3l3n2mt/variables/variables


INFO:tensorflow:Restoring parameters from /tmp/tmps3l3n2mt/variables/variables


...

INFO:tensorflow: tensor name: StatefulPartitionedCall:0, shape: (-1, 101), type: DT_FLOAT


INFO:tensorflow: tensor name: StatefulPartitionedCall:0, shape: (-1, 101), type: DT_FLOAT


INFO:tensorflow:Restoring parameters from /tmp/tmps3l3n2mt/variables/variables


INFO:tensorflow:Restoring parameters from /tmp/tmps3l3n2mt/variables/variables


5541984

copying commands for recovering workspace:


In [ ]:
!cp '/content/b0_model_lite_qed.tflite' '/content/drive/My Drive/Colab Notebooks'

In [2]:
!cp '/content/drive/My Drive/Colab Notebooks/b0_model_lite.tflite' /content

In [ ]:
!cp '/content/drive/My Drive/Colab Notebooks/socif/efficientnet_lite0_int8_1.tflite' /content

In [3]:
!cp '/content/drive/My Drive/Colab Notebooks/socif/lfw.tgz' /content
!cp '/content/drive/My Drive/Colab Notebooks/socif/lfw-names.txt' /content

In [4]:
!tar -zxvf lfw.tgz

Streaming output truncated to the last 5000 lines.
lfw/Luis_Sanchez/
lfw/Luis_Sanchez/Luis_Sanchez_0001.jpg
lfw/Juliette_Binoche/
lfw/Juliette_Binoche/Juliette_Binoche_0001.jpg
lfw/Joel_Todd/
lfw/Joel_Todd/Joel_Todd_0001.jpg
lfw/Allan_Wagner/
lfw/Allan_Wagner/Allan_Wagner_0001.jpg
lfw/Alejandro_Fernandez/
lfw/Alejandro_Fernandez/Alejandro_Fernandez_0001.jpg
lfw/Guillaume_Depardieu/
lfw/Guillaume_Depardieu/Guillaume_Depardieu_0001.jpg
lfw/Delphine_Chuillot/
lfw/Delphine_Chuillot/Delphine_Chuillot_0001.jpg
lfw/Paul_LeClerc/
lfw/Paul_LeClerc/Paul_LeClerc_0001.jpg
lfw/Eddie_Jordan/
lfw/Eddie_Jordan/Eddie_Jordan_0001.jpg
lfw/Lionel_Richie/
lfw/Lionel_Richie/Lionel_Richie_0001.jpg
lfw/Lionel_Richie/Lionel_Richie_0002.jpg
lfw/Jose_Viegas_Filho/
lfw/Jose_Viegas_Filho/Jose_Viegas_Filho_0001.jpg
lfw/Jose_Viegas_Filho/Jose_Viegas_Filho_0002.jpg
lfw/Roger_Mahony/
lfw/Roger_Mahony/Roger_Mahony_0001.jpg
lfw/Clay_Aiken/
lfw/Clay_Aiken/Clay_Aiken_0001.jpg
lfw/Clay_Aiken/Clay_Aiken_0002.jpg
lfw/Clay_Ai

In [3]:
import numpy as np
import tensorflow as tf
import cv2
import pandas as pd
# try run tflite
def test_tflite(path, dtype, dataset, names):
    # run tflite model
    interpreter = tf.compat.v1.lite.Interpreter(model_path=path)
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print(input_details)
    for m in range(len(dataset)):
        interpreter.set_tensor(input_details[0]['index'], [dataset[m].astype(dtype)])

        interpreter.invoke()

        # The function `get_tensor()` returns a copy of the tensor data.
        # Use `tensor()` in order to get a pointer to the tensor.
        output = []
        for i in range(3):
            output_data = interpreter.get_tensor(output_details[i]['index'])
            output.append(np.argmax(output_data))
        print(names.iloc[:,0][m], output)


In [5]:
import numpy as np
import pandas as pd
# This is for building the large dataset txt
result = []
source = pd.read_csv('lfw-names.txt', sep='\t', header=None)
for j, row in source.iterrows():
    filename = 'lfw/{}/{}_{}.jpg'
    name = row[0]
    amount = row[1]
    for i in range(int(amount)):
        number = str(i+1).zfill(4)
        result.append(filename.format(name, name, number))

with open('dataset_large.txt', 'w') as f:
    for i, l in enumerate(result):
        print('writing: {}'.format(l), end='\r')
        f.write('{}\n'.format(l))
print('\ndone')
print('Length of dataset: {}'.format(len(result)))

writing: lfw/Zydrunas_Ilgauskas/Zydrunas_Ilgauskas_0001.jpg
done
Length of dataset: 13233


In [4]:
# for output the 'ground truth' of the rknn model
csv_path = "dataset_large.txt"
dataset = pd.read_csv(csv_path, header=None)
large_ds = dataset[0].apply(lambda p: cv2.imread(p))
large_ds = large_ds.apply(lambda img: cv2.resize(img, (224, 224)))
large_ds = large_ds.apply(lambda img: cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
large_ds = np.array(large_ds.to_list(), dtype=np.float16)
test_tflite("b0_model_lite.tflite", dtype=np.float32, dataset=large_ds, names=dataset)

[{'name': 'input_dllm', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
lfw/AJ_Cook/AJ_Cook_0001.jpg [34, 1, 0]
lfw/AJ_Lamas/AJ_Lamas_0001.jpg [52, 0, 0]
lfw/Aaron_Eckhart/Aaron_Eckhart_0001.jpg [48, 0, 1]
lfw/Aaron_Guiel/Aaron_Guiel_0001.jpg [36, 0, 0]
lfw/Aaron_Patterson/Aaron_Patterson_0001.jpg [36, 0, 0]
lfw/Aaron_Peirsol/Aaron_Peirsol_0001.jpg [48, 0, 0]
lfw/Aaron_Peirsol/Aaron_Peirsol_0002.jpg [34, 0, 1]
lfw/Aaron_Peirsol/Aaron_Peirsol_0003.jpg [48, 1, 1]
lfw/Aaron_Peirsol/Aaron_Peirsol_0004.jpg [52, 0, 0]
lfw/Aaron_Pena/Aaron_Pena_0001.jpg [30, 0, 0]
lfw/Aaron_Sorkin/Aaron_Sorkin_0001.jpg [57, 0, 0]
lfw/Aaron_Sorkin/Aaron_Sorkin_0002.jpg [34, 0, 0]
lfw/Aaron_Tippin/Aaron_Tippin_0001.jpg [52

KeyboardInterrupt: ignored